아무런 Processing 없이 기존 변수들 RFECV

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
#import pandas_profiling

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
#from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import KFold
from sklearn.feature_selection import RFECV
from sklearn.preprocessing import LabelEncoder

from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC

from sklearn.metrics import roc_auc_score
from sklearn import preprocessing

import xgboost as xgb

from datetime import datetime
import lightgbm as lgb
#import multiprocessing
#import gc

In [2]:
def reduce_mem_usage(df):
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[: 3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min  and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
    
    end_mem = df.memory_usage().sum()/ 1024**2
    print('Memory usage after optimization is: {:.2f} MB, {:.1f}% reduction'.\
          format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    
    return df

In [3]:
import os
import gc
import sys
sys.path.append("../..")

In [4]:
os.getcwd()

'/home/biglake0987/ybigta_first_proj/EDA'

In [5]:
os.listdir()

['FinalPT_RFECV---1.ipynb',
 'test_transaction.csv',
 'train_transaction.csv',
 'FinalPT_RFECV-----1.ipynb',
 '.ipynb_checkpoints',
 'FinalPT_RFECV-1.ipynb',
 'test_identity.csv',
 'train_identity.csv',
 'sample_submission.csv',
 'sample.ipynb',
 'mintae1.ipynb',
 'V_cols(20190814).ipynb',
 '.gitkeep']

In [19]:
trans = pd.read_csv('train_transaction.csv', index_col='TransactionID')
test_trans = pd.read_csv('test_transaction.csv', index_col='TransactionID')

In [20]:
ident = pd.read_csv('train_identity.csv', index_col='TransactionID')
test_ident = pd.read_csv('test_identity.csv', index_col='TransactionID')

In [21]:
sample_sub = pd.read_csv('sample_submission.csv', index_col='TransactionID')

In [22]:
train = trans.merge(ident, how='left', left_index=True, right_index=True)
test = test_trans.merge(test_ident, how='left', left_index=True, right_index=True)

In [23]:
print(train.shape)
print(test.shape)

(590540, 433)
(506691, 432)


In [24]:
train.columns

Index(['isFraud', 'TransactionDT', 'TransactionAmt', 'ProductCD', 'card1',
       'card2', 'card3', 'card4', 'card5', 'card6',
       ...
       'id_31', 'id_32', 'id_33', 'id_34', 'id_35', 'id_36', 'id_37', 'id_38',
       'DeviceType', 'DeviceInfo'],
      dtype='object', length=433)

In [25]:
del trans, ident, test_trans, test_ident
gc.collect()

198

In [26]:
from tqdm import tqdm_notebook

In [27]:
for col in tqdm_notebook(train.columns): 
    if train[col].dtype == 'object':
        le = LabelEncoder()
        le.fit(list(train[col].astype(str).values) + list(test[col].astype(str).values))
        train[col] = le.transform(list(train[col].astype(str).values))

In [28]:
train = reduce_mem_usage(train)
test = reduce_mem_usage(test)

Memory usage after optimization is: 547.70 MB, 72.3% reduction
Memory usage after optimization is: 583.43 MB, 65.6% reduction


'TransactionDT'는 값들이 너무 커서 rfecv method로 감당 불가능한듯. 그래서 일단 제거

In [31]:
X = train.sort_values('V287').drop(['isFraud', 'TransactionDT'], axis=1)
y = train.sort_values('V287')['isFraud']

In [32]:
# RFECV does not support NaNs
X.fillna(-999, inplace=True)

In [33]:
print(train.shape)
print(test.shape)

(590540, 433)
(506691, 432)


In [34]:
params = {'num_leaves': 491,
          'min_child_weight': 0.03454472573214212,
          'feature_fraction': 0.3797454081646243,
          'bagging_fraction': 0.4181193142567742,
          'min_data_in_leaf': 106,
          'objective': 'binary',
          'max_depth': -1,
          'learning_rate': 0.006883242363721497,
          "boosting_type": "gbdt",
          "bagging_seed": 11,
          "metric": 'auc',
          "verbosity": -1,
          'reg_alpha': 0.3899927210061127,
          'reg_lambda': 0.6485237330340494,
          'random_state': 47
         }


In [35]:
clf = lgb.LGBMClassifier(**params)
rfe = RFECV(estimator=clf, step=10, cv=KFold(n_splits=5, shuffle=False), scoring='roc_auc', verbose=2)

In [36]:
rfe.fit(X, y)

Fitting estimator with 431 features.
Fitting estimator with 421 features.
Fitting estimator with 411 features.
Fitting estimator with 401 features.
Fitting estimator with 391 features.
Fitting estimator with 381 features.
Fitting estimator with 371 features.
Fitting estimator with 361 features.
Fitting estimator with 351 features.
Fitting estimator with 341 features.
Fitting estimator with 331 features.
Fitting estimator with 321 features.
Fitting estimator with 311 features.
Fitting estimator with 301 features.
Fitting estimator with 291 features.
Fitting estimator with 281 features.
Fitting estimator with 271 features.
Fitting estimator with 261 features.
Fitting estimator with 251 features.
Fitting estimator with 241 features.
Fitting estimator with 231 features.
Fitting estimator with 221 features.
Fitting estimator with 211 features.
Fitting estimator with 201 features.
Fitting estimator with 191 features.
Fitting estimator with 181 features.
Fitting estimator with 171 features.
F

Fitting estimator with 351 features.
Fitting estimator with 341 features.
Fitting estimator with 331 features.


RFECV(cv=KFold(n_splits=5, random_state=None, shuffle=False),
      estimator=LGBMClassifier(bagging_fraction=0.4181193142567742,
                               bagging_seed=11, boosting_type='gbdt',
                               class_weight=None, colsample_bytree=1.0,
                               feature_fraction=0.3797454081646243,
                               importance_type='split',
                               learning_rate=0.006883242363721497, max_depth=-1,
                               metric='auc', min_child_samples=20,
                               min_child_we....03454472573214212,
                               min_data_in_leaf=106, min_split_gain=0.0,
                               n_estimators=100, n_jobs=-1, num_leaves=491,
                               objective='binary', random_state=47,
                               reg_alpha=0.3899927210061127,
                               reg_lambda=0.6485237330340494, silent=True,
                               subsam

In [39]:
rfecv = rfe

In [40]:
print('최적 features 개수:', rfecv.n_features_)

최적 features 개수: 321


In [47]:
plt.figure(figsize=(400, 50))
plt.xlabel("선택된 Features 개수")
plt.ylabel("CV score")
plt.plot(range(1, len(rfecv.grid_scores_) + 1), rfecv.grid_scores_)
plt.show()

for col in X.columns[rfe.ranking_ == 1]:
    print(col)

In [48]:
final_V_cols = [col for col in X.columns[rfecv.ranking_ == 1]]
print(len(final_V_cols))
final_V_cols

321


['TransactionAmt',
 'ProductCD',
 'card1',
 'card2',
 'card3',
 'card4',
 'card5',
 'card6',
 'addr1',
 'addr2',
 'dist1',
 'dist2',
 'P_emaildomain',
 'R_emaildomain',
 'C1',
 'C2',
 'C4',
 'C5',
 'C6',
 'C7',
 'C8',
 'C9',
 'C10',
 'C11',
 'C12',
 'C13',
 'C14',
 'D1',
 'D2',
 'D3',
 'D4',
 'D5',
 'D6',
 'D7',
 'D8',
 'D9',
 'D10',
 'D11',
 'D12',
 'D13',
 'D14',
 'D15',
 'M2',
 'M3',
 'M4',
 'M5',
 'M6',
 'M7',
 'M8',
 'M9',
 'V2',
 'V3',
 'V4',
 'V5',
 'V7',
 'V12',
 'V13',
 'V15',
 'V19',
 'V20',
 'V23',
 'V24',
 'V25',
 'V26',
 'V29',
 'V30',
 'V33',
 'V34',
 'V35',
 'V36',
 'V37',
 'V38',
 'V39',
 'V40',
 'V43',
 'V44',
 'V45',
 'V47',
 'V48',
 'V49',
 'V51',
 'V52',
 'V53',
 'V54',
 'V55',
 'V56',
 'V57',
 'V58',
 'V59',
 'V60',
 'V61',
 'V62',
 'V63',
 'V64',
 'V66',
 'V67',
 'V69',
 'V70',
 'V73',
 'V74',
 'V75',
 'V76',
 'V77',
 'V78',
 'V79',
 'V80',
 'V81',
 'V82',
 'V83',
 'V85',
 'V86',
 'V87',
 'V90',
 'V91',
 'V93',
 'V94',
 'V96',
 'V97',
 'V98',
 'V99',
 'V100',
 'V1

In [49]:
del_cols = []
no_cols= []
for col in train.columns:

  if col not in final_V_cols:
    del_cols.append(col)
  else:
    no_cols.append(col)
print(len(no_cols));print(len(del_cols))
print(no_cols == final_V_cols)

321
112
True


In [50]:
del_cols

['isFraud',
 'TransactionDT',
 'C3',
 'M1',
 'V1',
 'V6',
 'V8',
 'V9',
 'V10',
 'V11',
 'V14',
 'V16',
 'V17',
 'V18',
 'V21',
 'V22',
 'V27',
 'V28',
 'V31',
 'V32',
 'V41',
 'V42',
 'V46',
 'V50',
 'V65',
 'V68',
 'V71',
 'V72',
 'V84',
 'V88',
 'V89',
 'V92',
 'V95',
 'V104',
 'V107',
 'V108',
 'V110',
 'V111',
 'V112',
 'V113',
 'V114',
 'V116',
 'V117',
 'V118',
 'V119',
 'V120',
 'V121',
 'V122',
 'V123',
 'V125',
 'V138',
 'V141',
 'V142',
 'V148',
 'V153',
 'V154',
 'V155',
 'V157',
 'V176',
 'V177',
 'V179',
 'V181',
 'V182',
 'V183',
 'V185',
 'V186',
 'V190',
 'V191',
 'V193',
 'V194',
 'V195',
 'V196',
 'V199',
 'V227',
 'V235',
 'V236',
 'V237',
 'V240',
 'V241',
 'V242',
 'V247',
 'V250',
 'V252',
 'V253',
 'V254',
 'V255',
 'V260',
 'V269',
 'V297',
 'V302',
 'V305',
 'V325',
 'V327',
 'V328',
 'V329',
 'V330',
 'V334',
 'V337',
 'V338',
 'V339',
 'id_04',
 'id_07',
 'id_08',
 'id_10',
 'id_12',
 'id_21',
 'id_22',
 'id_23',
 'id_24',
 'id_27',
 'id_28',
 'id_29']

In [51]:
del_cols.pop(0)

'isFraud'

In [52]:
len(del_cols)

111

In [53]:
train = train.drop(del_cols, axis=1)
test = test.drop(del_cols, axis=1)

In [54]:
print(train.shape); test.shape

(590540, 322)


(506691, 321)

In [ ]:
train.to_csv('rfecv_1_train.csv')
test.to_csv('rfecv_1_test.csv')

이제 rfecv-1을 가지고 XGBoost ! 